In [1]:
!pip -q install sentence-transformers

from sentence_transformers import SentenceTransformer
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch



# Load T5 / Scoring Logic

In [2]:
model = SentenceTransformer('sentence-t5-base')



modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

2_Dense/rust_model.ot:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [3]:
import pandas as pd
data = pd.read_excel("/kaggle/input/pred4444/pred4.xlsx")

In [4]:
data

,Unnamed: 0,Original Text,Prompt,Rewritten Text,output
0,0,The sun rises in the east and sets in the west.,Rewrite this as a romantic poem.,"In the east, where dreams take flight, the sun...",Rewrite the text in a more poetic and descrip...
1,1,Albert Einstein was a brilliant physicist who ...,Rewrite this as a conspiracy theory.,"Deep in the shadows of academia, whispers abou...",Change it into a conspiracy theorist's blog p...
2,2,Water boils at 100 degrees Celsius.,Rewrite this in the style of a scientific rese...,Empirical observations indicate that the phase...,vation: In the rewritten text the key elements...
3,3,The cat sat on the mat.,Rewrite this as a horror story.,"In the dimly lit room, a solitary figure watch...",Change it into a scene from a suspenseful mys...
4,4,Romeo and Juliet is a tragic love story writte...,Rewrite this as a clickbait headline.,You won't believe what happened to these star-...,Change it into a headline for a gossip magazine.
5,5,The Great Depression was a period of economic ...,Rewrite this as a children's bedtime story.,"Once upon a time, in a land not so far away, t...",Change it into a fairy tale.
6,6,Photosynthesis is the process by which plants ...,Rewrite this as a cooking recipe.,"To whip up a batch of energy, gather some gree...",Change it into a recipe for a delicious drink.
7,7,The Mona Lisa is a famous painting by Leonardo...,Rewrite this as a tweet from a modern art critic.,"Da Vinci's masterpiece, the Mona Lisa, transce...",Change it into a motivational quote.
8,8,The Earth orbits around the Sun.,Rewrite this as a text message from an alien s...,Observations confirm: planetary entity designa...,Rewrite the text in a more scientific tone.
9,9,Charles Dickens wrote 'A Tale of Two Cities'.,Rewrite this as a tweet from a literary fan ac...,Dive into the riveting tale woven by the inco...,Rewrite the sentence as a social media post.


In [5]:
def sharpened_cosine_similarity(vec1, vec2, exponent=3):
    cosine_similarity = torch.nn.functional.cosine_similarity(vec1, vec2, dim=0)
    return [cosine_similarity,cosine_similarity ** exponent]

In [12]:

data.reset_index(inplace=True,drop=True)
for index,value in enumerate(data['output']):
    pred = value
    pred = pred.replace("Change","Rewrite")
    original = data.loc[index,'Prompt']
    test_embedding = model.encode(str(pred), convert_to_tensor=True, show_progress_bar=False)
    original_embedding = model.encode(original, convert_to_tensor=True, show_progress_bar=False)
    cosine,exp = sharpened_cosine_similarity(test_embedding,original_embedding)
    data.loc[index,'cosine'] = cosine.item()
    data.loc[index,'exp'] = exp.item()

In [15]:
scores = list(data['exp'])

In [16]:
average_score = sum(scores) / len(scores)

In [17]:
average_score

0.6828169731990151

In [ ]:
output = "convert this to shanty"
original = "turn it into shanty for sailors"
test_embedding = model.encode(output, convert_to_tensor=True, show_progress_bar=False)
original_embedding = model.encode(original, convert_to_tensor=True, show_progress_bar=False)
cosine,exp = sharpened_cosine_similarity(test_embedding,original_embedding)
print("cosine is ",cosine.item())
print("exp ",exp.item())